In [1]:
import sys
sys.path.append('../src')
sys.path.append('..')

from main import generate_refined_data

from tqdm import tqdm
import pandas as pd

In [2]:
df = pd.read_csv('../data/data.csv')
df = df.sort_values(by='timestamp', ascending=True)
df = df.reset_index(drop=True)
df.head()

,timestamp,mac,map_id,rssi,site_id,type,x,y
0,1.723194e+09,6e55c33d4bf94fc912546aa53df0a83e6cd97bc542e861...,7445f317-78d8-4a93-add1-ac223c538c55,-55.300000,48b01af6-138b-465d-8996-bace824f5726,wifi,78.165060,49.166220
1,1.723194e+09,211dcd244bd609da55415aa490f81a19fda9563ec32425...,527aaffb-bc14-4013-b03c-d81bfde894bf,-62.000000,48b01af6-138b-465d-8996-bace824f5726,wifi,48.303600,22.940083
2,1.723194e+09,c806634ea9528565a397a6d8e2c6c11733892ed6c683d6...,529cbd22-3119-42b3-9bf9-086d108ed159,-47.550003,48b01af6-138b-465d-8996-bace824f5726,wifi,15.859675,18.266554
3,1.723194e+09,bf07a6f97d267a8841751725f1c270399910b53058b918...,162b807a-8c8b-4169-8c33-91882455b0a8,-68.200000,48b01af6-138b-465d-8996-bace824f5726,wifi,67.894010,37.696960
4,1.723194e+09,074b3881ba8aa8155093cf20719051392f85885c03bf8b...,caa07ab5-c4fb-4848-a8bf-57b482b429e6,-41.850000,48b01af6-138b-465d-8996-bace824f5726,wifi,52.415646,34.921520


In [3]:
# split into 5 minute intervals
# feed batches into main script -> generates refined data
# run script to turn refined data into occupancy stats
# run script to turn occupancy stats into density maps

In [4]:
timestamps = df['timestamp'].to_numpy()
cutoff_indices = [0]

prev_timestamp = timestamps[0]
for i, timestamp in enumerate(timestamps):

    if timestamp - prev_timestamp > 60*5:
        cutoff_indices.append(i)
        prev_timestamp = timestamp

    if len(cutoff_indices) == 2*6*6:
        break

print(cutoff_indices)

[0, 2026, 4659, 8703, 13386, 18911, 25192, 31964, 39636, 47386, 55804, 65327, 75290, 84768, 94282, 104142, 114803, 125615, 135843, 146291, 156441, 167336, 178184, 190182, 202459, 213763, 225234, 236686, 248250, 260796, 272795, 284573, 296370, 306690, 316883, 326793, 334762, 342985, 349319, 356786, 363717, 371538, 378697, 386281, 393409, 400721, 407162, 412832, 418414, 424243, 429258, 434014, 439196, 444140, 449621, 454811, 460450, 466188, 471152, 477215, 482019, 487248, 492569, 497481, 502732, 507914, 513612, 518892, 523859, 529061, 534747, 539526]


In [5]:
batches = []

for i in tqdm(range(len(cutoff_indices))):
    if i == len(cutoff_indices) - 1:
        break

    batch = df.iloc[cutoff_indices[i]:cutoff_indices[i+1]]
    batches.append(batch)

 99%|█████████▊| 71/72 [00:00<00:00, 43115.04it/s]


In [6]:
print(len(batches))
print(len(batches[3]))

71
4683


In [7]:
for i, batch in enumerate(tqdm(batches)):
    if i == 0:
        generate_refined_data(batch, first_batch=True)
    else:
        generate_refined_data(batch)

  1%|▏         | 1/71 [00:00<00:27,  2.50it/s]

Loading data took 0.31 seconds
Processing data took 0.05 seconds
Writing data took 0.01 seconds


/Users/carl/Desktop/ict/wifi-tracking2/test/../src/device.py:66: RuntimeWarning: invalid value encountered in scalar divide
  z = radius_to_prev / prev_error
  3%|▎         | 2/71 [00:00<00:31,  2.17it/s]

Loading data took 0.33 seconds
Processing data took 0.08 seconds
Writing data took 0.06 seconds
Loading data took 0.33 seconds
Processing data took 0.15 seconds
Writing data took 0.05 seconds


/Users/carl/Desktop/ict/wifi-tracking2/test/../src/device.py:66: RuntimeWarning: divide by zero encountered in scalar divide
  z = radius_to_prev / prev_error
  4%|▍         | 3/71 [00:01<00:34,  1.96it/s]

...

Writing data took 2.18 seconds
Loading data took 0.87 seconds
Processing data took 1.29 seconds


 99%|█████████▊| 70/71 [03:53<00:04,  4.41s/it]

Writing data took 2.26 seconds
Loading data took 0.98 seconds
Processing data took 1.34 seconds


100%|██████████| 71/71 [03:57<00:00,  3.35s/it]

Writing data took 2.09 seconds


In [4]:
timestamps = df['timestamp'].to_numpy()
batch_1 = df[df['timestamp'] < timestamps[0] + 60 * 5]
batch_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2024 entries, 0 to 2023
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  2024 non-null   float64
 1   mac        2024 non-null   object 
 2   map_id     2024 non-null   object 
 3   rssi       2024 non-null   float64
 4   site_id    2024 non-null   object 
 5   type       2024 non-null   object 
 6   x          2024 non-null   float64
 7   y          2024 non-null   float64
dtypes: float64(4), object(4)
memory usage: 142.3+ KB


In [5]:
batch_2 = df[(df['timestamp'] >= timestamps[0] + 60 * 5) & (df['timestamp'] < timestamps[0] + 60 * 10)]
batch_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2621 entries, 2024 to 4644
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  2621 non-null   float64
 1   mac        2621 non-null   object 
 2   map_id     2621 non-null   object 
 3   rssi       2621 non-null   float64
 4   site_id    2621 non-null   object 
 5   type       2621 non-null   object 
 6   x          2621 non-null   float64
 7   y          2621 non-null   float64
dtypes: float64(4), object(4)
memory usage: 184.3+ KB


In [6]:
generate_refined_data(batch_1, first_batch=True)

Loading data took 0.34 seconds
Processing data took 0.05 seconds
Writing data took 0.01 seconds


In [7]:
generate_refined_data(batch_2, first_batch=False)

{'6e55c33d4bf94fc912546aa53df0a83e6cd97bc542e86141c4827ecb6ae4ab8a': <device.Device object at 0x1309eb250>, '211dcd244bd609da55415aa490f81a19fda9563ec324256fc2a956d11e119a28': <device.Device object at 0x13674bc50>, 'c806634ea9528565a397a6d8e2c6c11733892ed6c683d606dabf6942a9031628': <device.Device object at 0x1309eb510>, 'bf07a6f97d267a8841751725f1c270399910b53058b918fd2627a052462fc193': <device.Device object at 0x1309e9b10>, '074b3881ba8aa8155093cf20719051392f85885c03bf8bd5a23337f6bc67ea3c': <device.Device object at 0x1309e8350>, '71429fde930618013134ee44b3772509984ef0bca4d5c4e184f6fb83bc2badee': <device.Device object at 0x1309eb2d0>, '4da56d39adfea01ac46c912bf732c08d271a6543c01fe49254c52d759334c644': <device.Device object at 0x1309eab50>, 'c9aa0012377f51e02779d5ed023d9c01a773ff9055e7b093e77088ab2cc98975': <device.Device object at 0x1309ea390>, '30cdf56e4a5e86369a59e0ea9970d979a9ecf963916e4661690edc203fd75b3f': <device.Device object at 0x1309e9b50>, '7d5871548129928e87a5293826512153244

/Users/carl/Desktop/ict/wifi-tracking2/test/../src/device.py:66: RuntimeWarning: invalid value encountered in scalar divide
  z = radius_to_prev / prev_error
